In [ ]:
!pip install pycaret

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import RobustScaler, StandardScaler
from pycaret.regression import *
from catboost import CatBoostRegressor
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.impute import KNNImputer

In [ ]:
train = pd.read_csv('../input/parkingdata/train.csv')
test = pd.read_csv('../input/parkingdata/test.csv')
submission = pd.read_csv('../input/parkingdata/sample_submission.csv')
info = pd.read_csv('../input/parkingdata/age_gender_info.csv')

In [ ]:
train.isnull().sum()

In [ ]:
train['전용면적'] = train['전용면적']//10*10
test['전용면적'] = test['전용면적']//10*10

idx = train[train['전용면적']>100].index
train.loc[idx, '전용면적'] = 100
idx = test[test['전용면적']>100].index
test.loc[idx, '전용면적'] = 100

idx = train[train['전용면적']<15].index
train.loc[idx, '전용면적'] = 15
idx = test[test['전용면적']<15].index
test.loc[idx, '전용면적'] = 15

In [ ]:
train.head()

In [ ]:
#### 결측치 처리
train.loc[train.임대보증금=='-', '임대보증금'] = np.nan
test.loc[test.임대보증금=='-', '임대보증금'] = np.nan
train['임대보증금'] = train['임대보증금'].astype(float)
test['임대보증금'] = test['임대보증금'].astype(float)

train.loc[train.임대료=='-', '임대료'] = np.nan
test.loc[test.임대료=='-', '임대료'] = np.nan
train['임대료'] = train['임대료'].astype(float)
test['임대료'] = test['임대료'].astype(float)

train[['임대보증금', '임대료']] = train[['임대보증금', '임대료']].fillna(0)
test[['임대보증금', '임대료']] = test[['임대보증금', '임대료']].fillna(0)

cols = ['도보 10분거리 내 지하철역 수(환승노선 수 반영)', '도보 10분거리 내 버스정류장 수']
train[cols] = train[cols].fillna(0)
test[cols] = test[cols].fillna(0)

#### 자격유형

test.loc[test.단지코드.isin(['C2411']) & test.자격유형.isnull(), '자격유형'] = 'A'
test.loc[test.단지코드.isin(['C2253']) & test.자격유형.isnull(), '자격유형'] = 'C'


#### 중복 처리
train = train.drop_duplicates()
test = test.drop_duplicates()

unique_cols = ['총세대수', '지역', '공가수', 
               '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
               '도보 10분거리 내 버스정류장 수',
               '단지내주차면수', '등록차량수']
train_agg = train.set_index('단지코드')[unique_cols].drop_duplicates()
test_agg = test.set_index('단지코드')[[col for col in unique_cols if col!='등록차량수']].drop_duplicates()

In [ ]:
#area_columns = []
#for area in train['전용면적'].unique():
#    area_columns.append(f'면적_{area}')

In [ ]:
train.columns

In [ ]:
"""
from tqdm import tqdm

new_train = pd.DataFrame()
new_test = pd.DataFrame()

columns = ['단지코드','총세대수', '지역', '공가수', 
               '도보 10분거리 내 지하철역 수(환승노선 수 반영)',
               '도보 10분거리 내 버스정류장 수',
               '단지내주차면수']

for i, code in tqdm(enumerate(train['단지코드'].unique())):
    temp = train[train['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_train.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_train.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
    
    new_train.loc[i, '등록차량수'] = temp.loc[0, '등록차량수']
    
for i, code in tqdm(enumerate(test['단지코드'].unique())):
    temp = test[test['단지코드']==code]
    temp.index = range(temp.shape[0])
    for col in columns:
        new_test.loc[i, col] = temp.loc[0, col]
    
    for col in area_columns:
        area = float(col.split('_')[-1])
        new_test.loc[i, col] = temp[temp['전용면적']==area]['전용면적별세대수'].sum()
        """

In [ ]:
#new_train = new_train.set_index('단지코드')
#new_test = new_test.set_index('단지코드')

In [ ]:
def reshape_cat_features(data, cast_col, value_col):
    res = data.drop_duplicates(['단지코드', cast_col]).assign(counter=1).pivot(index='단지코드', columns=cast_col, values=value_col).fillna(0)
    res.columns.name = None
    res = res.rename(columns={col:cast_col+'_'+col for col in res.columns})
    return res

train.loc[train.공급유형.isin(['공공임대(5년)', '공공분양', '공공임대(10년)', '공공임대(분납)']), '공급유형'] = '공공임대(5년/10년/분납/분양)'
test.loc[test.공급유형.isin(['공공임대(5년)', '공공분양', '공공임대(10년)', '공공임대(분납)']), '공급유형'] = '공공임대(5년/10년/분납/분양)'
train.loc[train.공급유형.isin(['장기전세', '국민임대']), '공급유형'] = '국민임대/장기전세'
test.loc[test.공급유형.isin(['장기전세', '국민임대']), '공급유형'] = '국민임대/장기전세'

train.loc[train.자격유형.isin(['J', 'L', 'K', 'N', 'M', 'O']), '자격유형'] = '행복주택_공급대상'
test.loc[test.자격유형.isin(['J', 'L', 'K', 'N', 'M', 'O']), '자격유형'] = '행복주택_공급대상'

train.loc[train.자격유형.isin(['H', 'B', 'E', 'G']), '자격유형'] = '국민임대/장기전세_공급대상'
test.loc[test.자격유형.isin(['H', 'B', 'E', 'G']), '자격유형'] = '국민임대/장기전세_공급대상'

train.loc[train.자격유형.isin(['C', 'I', 'F']), '자격유형'] = '영구임대_공급대상'
test.loc[test.자격유형.isin(['C', 'I', 'F']), '자격유형'] = '영구임대_공급대상'

X_train = pd.concat([train_agg,
                       reshape_cat_features(data=train, cast_col='임대건물구분', value_col='counter'),
                       reshape_cat_features(data=train, cast_col='공급유형', value_col='counter'),
                       reshape_cat_features(data=train, cast_col='자격유형', value_col='counter')], axis=1)

X_test = pd.concat([test_agg,
                       reshape_cat_features(data=test, cast_col='임대건물구분', value_col='counter'),
                       reshape_cat_features(data=test, cast_col='공급유형', value_col='counter'),
                       reshape_cat_features(data=test, cast_col='자격유형', value_col='counter')], axis=1)

In [ ]:
X_train = X_train.drop(columns = ['임대건물구분_아파트'])
X_test = X_test.drop(columns = ['임대건물구분_아파트'])

In [ ]:
X_train = pd.merge(X_train, info, left_on= ["지역"], right_on= ["지역"], how='left')
X_test = pd.merge(X_test, info, left_on= ["지역"], right_on= ["지역"], how='left')

In [ ]:
print(X_train.shape,train.shape)
X_train.columns

In [ ]:
# divide
for num in range(10,110,10):
    X_train[str(num)+'대'] = X_train[str(num)+'대(여자)'] + X_train[str(num)+'대(남자)']
    X_test[str(num)+'대'] = X_test[str(num)+'대(여자)'] + X_test[str(num)+'대(남자)']
    if num == 10:
        X_train[str(num)+'대(미만)'] = X_train[str(num)+'대미만(여자)'] + X_train[str(num)+'대미만(남자)']
        X_test[str(num)+'대(미만)'] = X_test[str(num)+'대미만(여자)'] + X_test[str(num)+'대미만(남자)']
    
for i in range(10,110,10):
    X_train.drop(str(i)+'대(여자)',axis=1,inplace=True)
    X_train.drop(str(i)+'대(남자)',axis=1,inplace=True)
    X_test.drop(str(i)+'대(여자)',axis=1,inplace=True)
    X_test.drop(str(i)+'대(남자)',axis=1,inplace=True)
    if i == 10:
        X_train.drop(str(i)+'대미만(여자)',axis=1,inplace=True)
        X_test.drop(str(i)+'대미만(여자)',axis=1,inplace=True)
        X_train.drop(str(i)+'대미만(남자)',axis=1,inplace=True)
        X_test.drop(str(i)+'대미만(남자)',axis=1,inplace=True)
        
# merge
X_train['10대~20대'] = X_train['10대(미만)'] + X_train['10대'] + X_train['20대']
X_train['30대~50대'] = X_train['30대'] + X_train['40대'] + X_train['50대']
X_train['60이상'] = X_train['60대'] + X_train['70대'] + X_train['80대'] + X_train['90대'] + X_train['100대']

X_test['10대~20대'] = X_test['10대(미만)'] + X_test['10대'] + X_test['20대']
X_test['30대~50대'] = X_test['30대'] + X_test['40대'] + X_test['50대']
X_test['60이상'] = X_test['60대'] + X_test['70대'] + X_test['80대'] + X_test['90대'] + X_test['100대']

X_train.drop(['10대(미만)','10대','20대','30대','40대','50대','60대','70대','80대','90대','100대'],axis=1,inplace=True)
X_test.drop(['10대(미만)','10대','20대','30대','40대','50대','60대','70대','80대','90대','100대'],axis=1,inplace=True)

In [ ]:
X_train.isnull().sum(),X_test.isnull().sum()

In [ ]:
%%time
encoding_features = ['지역', '도보 10분거리 내 지하철역 수(환승노선 수 반영)']
for f in encoding_features:
    mapping = X_train.groupby([f])['등록차량수'].agg(['mean','median','std'])
    mapping_values = []
    for l in X_train[f].values:
        mapping_values.extend([mapping.loc[l].values])
    bincount = pd.DataFrame(mapping_values, columns = [f+'_mean', f+'_median', f+'_std'], index = X_train.index)
    X_train = pd.concat([X_train,bincount], axis= 1).drop(columns = [f])

    mapping_values = []
    for l in X_test[f].values:
        mapping_values.extend([mapping.loc[l].values])
    bincount = pd.DataFrame(mapping_values, columns = [f+'_mean', f+'_median', f+'_std'], index = X_test.index)
    X_test = pd.concat([X_test,bincount], axis= 1).drop(columns = [f])

In [ ]:
#X_train = X_train.drop(X_train.columns[X_train.columns.str.contains('여자') | X_train.columns.str.contains('남자')],axis=1)
#X_test = X_test.drop(X_test.columns[X_test.columns.str.contains('여자') | X_test.columns.str.contains('남자')],axis=1)

In [ ]:
X_train.columns

In [ ]:
scaler = RobustScaler()
loc_f = [    '지역_mean', 
             '지역_median',
             '지역_std']
X_train.loc[:, loc_f] = scaler.fit_transform(X_train[loc_f])
X_test.loc[:, loc_f] = scaler.transform(X_test[loc_f])

In [ ]:
scailing_features = X_train.drop(X_train.columns[X_train.columns.str.contains('지역')| X_train.columns.str.contains('유형') | X_train.columns.str.contains('등록차량수')],axis=1).columns.tolist()
scailing_features

In [ ]:
scaler = StandardScaler()
X_train.loc[:, scailing_features] = scaler.fit_transform(X_train[scailing_features])
X_test.loc[:, scailing_features] = scaler.transform(X_test[scailing_features])

In [ ]:
corr = X_train[X_train.columns].corr().abs()
corr.loc['등록차량수'].sort_values(ascending=False)

In [ ]:
X = X_train.copy()
X = X.loc[:,corr.index[:]]
X.drop(['등록차량수'],axis=1, inplace=True)
feature_names = X.columns.to_list()
y = X_train['등록차량수']

- pycret

In [ ]:
reg = setup(X_train, 
            preprocess = False, # True로 설정되면, 자체적인 Feature Engineering을 추가로 진행해 Predict가 불가능해진다.
            train_size = 0.99,  # 우리는 전체 데이터를 학습해 test를 예측하는게 목표이기 때문에, 0.999로 설정한다.
            target = '등록차량수', # 목표 변수는 등록 차량 수 이다.
            silent = True, # 엔터를 누르기 귀찮다. 궁금하면 풀어보세요
            use_gpu = False, # GPU가 있으면 사용하세요 (Cat BOost 속도 향상)
            numeric_features=list(X_train.drop(columns = ['등록차량수']).columns), # 모든 변수가 숫자로써의 의미가 있다.
            session_id = 2022,
            fold_shuffle = True
            )

In [ ]:
top3 = compare_models(n_select = 3, sort = 'MAE')

In [ ]:
models = []
for m in top3:
    models.append(tune_model(m, 
                             optimize = 'MAE', 
                             choose_better = True,
                            n_iter = 30))

In [ ]:
X_test = X_test[feature_names]
voting = blend_models(models, optimize = 'MAE')
voting = tune_model(voting, 
                 optimize = 'MAE', 
                 choose_better = True,
                 n_iter = 30)

voting = finalize_model(voting)
layer1_pred = voting.predict(X_test)
submission['num'] = layer1_pred
submission.to_csv('baseline3model_last.csv', index=False)

In [ ]:
submission

In [ ]:
submission[submission['num']<0]

- Hyper params Tuning

In [ ]:
def objective(trial: Trial) -> float:
    params_cat = {
        "random_state": 42,
        "learning_rate": 0.01,
        "n_estimators": 10000,
        "verbose" : 1,
        "objective" : "MAE",
        "max_depth": trial.suggest_int("max_depth", 1, 16),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.8, 1.0),
        "subsample": trial.suggest_float("subsample", 0.3, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "max_bin": trial.suggest_int("max_bin", 200, 500),
    }
    
    X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2)

    model = CatBoostRegressor(**params_cat)
    model.fit(
        X_tr,
        y_tr,
        eval_set=[(X_tr, y_tr), (X_val, y_val)],
        early_stopping_rounds=10,
        verbose=False,
    )

    cat_pred = model.predict(X_val)
    log_score = mean_absolute_error(y_val, cat_pred)
    
    return log_score

In [ ]:
sampler = TPESampler(seed=42)
study = optuna.create_study(
    study_name="cat_opt",
    direction="minimize",
    sampler=sampler,
)
study.optimize(objective, n_trials=10)
print("Best Score:", study.best_value)
print("Best trial:", study.best_trial.params)

In [ ]:
cat_p = study.best_trial.params
cat = CatBoostRegressor(**cat_p)

- StratifiedK-Fold for Regression

In [ ]:
X_test = X_test[feature_names]
X.shape,y.shape,X_test.shape

In [ ]:
y_cat = pd.cut(y, 10, labels=range(10))
skf = StratifiedKFold(5)

preds = []
for tr_id, val_id in skf.split(X, y_cat) : 
    X_tr = X.iloc[tr_id]
    y_tr = y.iloc[tr_id]
    
    cat.fit(X_tr, y_tr, verbose = 0)
    
    pred = cat.predict(X_test)
    preds.append(pred)
cat_pred = np.mean(preds, axis = 0)

In [ ]:
submission['num'] = cat_pred
submission.to_csv('catboost[hyper_best]_last.csv', index=False)

In [ ]:
submission